In [1]:
import turtle
import random
import numpy as np
from keras import Sequential
from collections import deque
from keras.layers import Dense
import matplotlib.pyplot as plt
from keras.optimizers import Adam
import pickle
import tensorflow as tf
import datetime, os
import os
import sys
import time
import json
import psutil
import copy
grid = np.linspace(-240, 240, 49)
%load_ext tensorboard

In [2]:
grid

array([-240., -230., -220., -210., -200., -190., -180., -170., -160.,
       -150., -140., -130., -120., -110., -100.,  -90.,  -80.,  -70.,
        -60.,  -50.,  -40.,  -30.,  -20.,  -10.,    0.,   10.,   20.,
         30.,   40.,   50.,   60.,   70.,   80.,   90.,  100.,  110.,
        120.,  130.,  140.,  150.,  160.,  170.,  180.,  190.,  200.,
        210.,  220.,  230.,  240.])

In [3]:
def monitor_memory():
    process = psutil.Process()
    memory_info = process.memory_info()
    total_memory = psutil.virtual_memory().total
    
    used_percent = (memory_info.rss / total_memory) * 100
    disk_usage = psutil.disk_usage("/")
    percent_used = disk_usage.percent
    print(f"RAM Usage: {used_percent:.2f}%")
    print(f"Disk Space Used: {percent_used:.2f}%")

In [4]:
class TurtleGame():
    
    def __init__(self):
        
        self.done = False
        self.reward = 0
        self.count = 0
        self.gameover = False
        self.level = 1
        self.n = 0
        
        #sfondo
        
        self.win = turtle.Screen()
        self.win.title('Bad')
        self.win.bgcolor('Black')
        self.win.tracer(0)
        self.win.setup(width=600, height = 600)
        
        #turtle
        
        self.animal = turtle.Turtle()
        self.animal.shape('turtle')
        self.animal.direction = 'stop'
        self.animal.color('yellow')
        self.animal.shapesize(stretch_wid=1, stretch_len=1)
        self.animal.penup()
        self.animal.goto(random.choice(grid),random.choice(grid))
        
        #food
        self.foods = []
        
        for _ in range(3):
            self.food = turtle.Turtle()
            self.food.shape('circle')
            self.food.penup()
            self.food.color('orange')
            
            self.food.goto(random.choice(grid),random.choice(grid))
            
            self.foods.append(self.food)
        
        #commands
        
        self.win.listen()
        self.win.onkeypress(self.animal_right, 'Right')   
        self.win.onkeypress(self.animal_left, 'Left')
        self.win.onkeypress(self.animal_up, 'Up')   
        self.win.onkeypress(self.animal_down, 'Down')
        
        
        #distance
        
        self.distance = np.abs(self.animal.distance(self.food.xcor(), self.food.ycor()))
        
        #lives and score
        
        self.lives = 1
        self.score = 0
        
        
        
        self.pen = turtle.Turtle()
        self.pen.speed(0)
        self.pen.color('blue')
        self.pen.penup()
        self.pen.goto(0,230)
        self.pen.pendown()
        self.pen.write('Score: {}  Lives:'.format(self.score, self.lives), align = 'center', font=('Courier', 20))
        self.pen.hideturtle()
        


        
        #enemies
        
        self.enemies = []
        for x in range(25):
            self.enemy = turtle.Turtle()
            self.enemy.penup()
            self.enemy.color('red')
            self.enemy.shape('circle')
            self.enemy.shapesize(stretch_wid=0.5, stretch_len=0.5)
            self.enemy.speed = 10
            self.enemy.goto(random.choice(grid),random.choice(grid))
            self.enemies.append(self.enemy)
            self.enemy.frame_count = 0
            self.enemy.direction = random.choice(['up', 'down', 'left', 'right'])
            self.enemy.dir = 0
            if self.enemy.direction == 'up':
                self.enemy.dir = 1.
            if self.enemy.direction == 'down':
                self.enemy.dir = -1.
            if self.enemy.direction == 'left':
                self.enemy.dir = 0.5  
            if self.enemy.direction == 'right':
                self.enemy.dir = -0.5
                
    #movement
    
    def movement(self):
        
        if self.animal.direction == 'right':
            
            x = self.animal.xcor() 

            if self.animal.heading() != 0.0:
                self.animal.setheading(0.0)

            if x < 240:

                self.animal.setx(x+10)
                
        if self.animal.direction == 'left':
            
            x = self.animal.xcor()   

            if self.animal.heading() != 180.0:
                self.animal.setheading(180.0)

            if x > -240:
                self.animal.setx(x-10)  

        if self.animal.direction == 'up':
            
            y = self.animal.ycor()

            if self.animal.heading() != 90.0:
                self.animal.setheading(90.0)

            if y < 240:
                self.animal.sety(y+10)
            
        if self.animal.direction == 'down':
            
            y = self.animal.ycor()

            if self.animal.heading() != 270.0:
                self.animal.setheading(270.0)

            if y > -240:
                
                self.animal.sety(y-10)
                
    
    def move_enemies(self):
        
        for enemy in self.enemies:
            
            if enemy.frame_count == 0:
                enemy.direction = random.choice(['up', 'down', 'left', 'right'])
                enemy.frame_count = random.randint(10, 20)
            else:
                enemy.frame_count -= 1

            if enemy.direction == 'up':
                if enemy.ycor() + enemy.speed > 240:
                    enemy.direction = 'down'
                else:
                    enemy.sety(enemy.ycor() + enemy.speed)
            elif enemy.direction == 'down':
                if enemy.ycor() - enemy.speed < -240:
                    enemy.direction = 'up'
                else:
                    enemy.sety(enemy.ycor() - enemy.speed)
            elif enemy.direction == 'left':
                if enemy.xcor() - enemy.speed < -240:
                    enemy.direction = 'right'
                else:
                    enemy.setx(enemy.xcor() - enemy.speed)
            elif enemy.direction == 'right':
                if enemy.xcor() + enemy.speed > 240:
                    enemy.direction = 'left'
                else:
                    enemy.setx(enemy.xcor() + enemy.speed)
        
    def animal_right(self):
        
        self.animal.direction = 'right'
        
    def animal_left(self):
        
        self.animal.direction = 'left'

    def animal_down(self):
        
        self.animal.direction = 'down'
            
    def animal_up(self):
        
        self.animal.direction = 'up'
    
    #window update
    
    def run_frame(self):
        

        env.win.update()

        for food in env.foods:

            if env.animal.distance(food) == 0:
                #env.foods.remove(food)
                food.goto(random.choice(grid),random.choice(grid))
                #env.foods.append(food)
                env.count += 1
                env.score += 1
                env.pen.clear()
                env.pen.write('Score: {}  Lives: {} '.format(env.score, env.lives), align = 'center', font=('Courier', 20))

    #        if env.count == 6:
    #            env.done = True
    #            env.reset()
    #            env.level += 1
    #            env.count = 0
    #            env.score = 0
    #           env.lives = 1
    #            env.pen.clear()
    #            env.pen.write('Score: {}  Lives: {} '.format(env.score, env.lives), align = 'center', font=('Courier', 20))

        for enemy in env.enemies:

            if env.animal.distance(enemy) == 0 and env.lives != 0:

                env.lives -= 1
                print('Score: ', env.score)
                env.pen.clear()
                env.pen.write('Score: {}  Lives: {}'.format(env.score, env.lives), align = 'center', font=('Courier', 20))
                time.sleep(1)
                env.animal.goto(random.choice(grid),random.choice(grid))

        if env.lives == 0:

            env.done = True
            env.reset_good()
            env.level = 1
            env.count = 0
            env.score = 0
            env.lives = 1
            game_over = turtle.Turtle()
            game_over.speed(0)
            game_over.color('yellow')
            game_over.penup()
            game_over.hideturtle()
            game_over.goto(0, 0)
            game_over.write('Game Over', align='center', font=('Courier', 40, 'bold'))
            time.sleep(2)
            game_over.clear()
            env.pen.clear()
            env.pen.write('Score: {}  Lives: {}'.format(env.score, env.lives), align = 'center', font=('Courier', 20))
            del game_over
        


    #AI movement
    
    def reset(self):
        
        dist_in = []
        
            
        for food in self.foods:
            food.goto(random.choice(grid),random.choice(grid))


        self.animal.goto(random.choice(grid),random.choice(grid))
        #self.foods = []
        
#        for _ in range(3):
#            
#            self.food = turtle.Turtle()
#            self.food.shape('circle')
#            self.food.penup()
#            self.food.color('orange')
#            
#            self.food.goto(random.choice(grid),random.choice(grid))
#            
#            self.foods.append(self.food)
            
        for food in self.foods:
            dist_in.append(np.abs(self.animal.distance(food.xcor(), food.ycor())))
            
        ind_f = np.argmin(dist_in)
        

        state = [self.animal.xcor()/240, self.animal.ycor()/240,
                (self.animal.xcor()-self.foods[ind_f].xcor())/240,
                (self.animal.ycor()-self.foods[ind_f].ycor())/240]
        
        distin_en = []
        
        for en in self.enemies:
            distin_en.append(np.abs(self.animal.distance(en.xcor(), en.ycor())))
            
        indices = sorted(range(len(distin_en)), key=lambda i: distin_en[i])[:4]
        for ind in indices:
            if (np.abs(self.animal.xcor()-self.enemies[ind].xcor()) < 51. and np.abs(self.animal.ycor()-self.enemies[ind].ycor())< 51.):
                
                state.extend([(self.animal.xcor()-self.enemies[ind].xcor())/240, 
                            (self.animal.ycor()-self.enemies[ind].ycor())/240, self.enemies[ind].dir])
            else:
                state.extend([1000, 1000, 1000])
            
        del distin_en
        del dist_in
        
        return state
    
    
    def reset_good(self):
        
        dist_in = []
        
            
        for food in self.foods:
            food.goto(random.choice(grid),random.choice(grid))


        self.animal.goto(random.choice(grid),random.choice(grid))
        #self.foods = []
        
#        for _ in range(3):
#            
#            self.food = turtle.Turtle()
#            self.food.shape('circle')
#            self.food.penup()
#            self.food.color('orange')
#            
#            self.food.goto(random.choice(grid),random.choice(grid))
#            
#            self.foods.append(self.food)
            
        for food in self.foods:
            dist_in.append(np.abs(self.animal.distance(food.xcor(), food.ycor())))
            
        ind_f = np.argmin(dist_in)
        

        state = [self.animal.xcor()/240, self.animal.ycor()/240,
                (self.animal.xcor()-self.foods[ind_f].xcor())/240,
                (self.animal.ycor()-self.foods[ind_f].ycor())/240]
        
        distin_en = []
        
        for en in self.enemies:
            distin_en.append(np.abs(self.animal.distance(en.xcor(), en.ycor())))
            
        indices = sorted(range(len(distin_en)), key=lambda i: distin_en[i])[:4]
        for ind in indices:
            
            state.extend([(self.animal.xcor()-self.enemies[ind].xcor())/240, 
                        (self.animal.ycor()-self.enemies[ind].ycor())/240,
                         self.enemies[ind].dir/240])
        del distin_en
        del dist_in
        
        return state
    
    
    def step(self, action):
        
        self.done = 0
        self.reward = 0
        dist_in = []
        dist_fin = []
        distin_en = []
        distfin_en = []
        x_in = []
        y_in = []
        
        rew_walls = 2
        rew_food_collect = 10.
        rew_collision = -20.
        rew_seek_food = 3
        rew_avoid_food = -3
        rew_seek_closer_enemy_far = -1
        rew_avoid_closer_enemy_far = +1
        rew_seek_closer_enemy_close = -3
        rew_avoid_closer_enemy_close = +3
        rew_seek_enemy_far = -1
        rew_avoid_enemy_far= +1
        rew_seek_enemy_close = -2
        rew_avoid_enemy_close= +2
   
        
        
        for food in self.foods:
            dist_in.append(np.abs(self.animal.distance(food.xcor(), food.ycor())))
        
        
        for en in self.enemies:
            distin_en.append(np.abs(self.animal.distance(en.xcor(), en.ycor())))
            x_in.append(en.xcor())
            y_in.append(en.ycor())
            en.color('red')

        if action == 0:
            self.animal_left()
            
            
            if self.animal.xcor() == -240:
                self.reward -= rew_walls
                
        if action == 1:
            self.animal_right()
            
            
            if self.animal.xcor() == +240:
                self.reward -= rew_walls
            
        if action == 2:
            self.animal_down()
            
            
            if self.animal.ycor() == -240:
                self.reward -= rew_walls
                
        if action == 3:
            self.animal_up()
        
            if self.animal.ycor() == +240:
                self.reward -= rew_walls
                
        ind_f = np.argmin(dist_in)
        ind_e = np.argmin(distin_en)        
        #self.enemies[ind_e].color('green')
        x_en = self.enemies[ind_e].xcor()
        y_en = self.enemies[ind_e].ycor()
        
        indices = sorted(range(len(distin_en)), key=lambda i: distin_en[i])[:4]
        self.movement()
        self.move_enemies()

        
        for food in self.foods:
            dist_fin.append(np.abs(self.animal.distance(food.xcor(), food.ycor())))
            
        for en in self.enemies:
            distfin_en.append(np.abs(self.animal.distance(en.xcor(), en.ycor())))
            
        for d in dist_fin:
            if d == 0.:
                self.reward += rew_food_collect
                #self.n = 0
                
        for d in distfin_en:
            if d == 0.:
                self.reward += rew_collision
                self.done = 1.
                
        
        
        
        if dist_fin[ind_f] < dist_in[ind_f]:
            
            self.reward += rew_seek_food
        else:
            self.reward += rew_avoid_food

       #print('la distanza iniziale era', distin_en[ind_e])
        #print('la distanza finale è', self.animal.distance(x_en, y_en))
        
        if 26. <= distin_en[ind_e] <= 65.:
            
            
            if np.abs(self.animal.distance(x_en, y_en)) < distin_en[ind_e]:
            
                self.reward += rew_seek_closer_enemy_far
               
            else:

                self.reward += rew_avoid_closer_enemy_far
                

                
        if 0 <=distin_en[ind_e] <= 25:
            

            if np.abs(self.animal.distance(x_in[ind_e], y_in[ind_e])) <= distin_en[ind_e]:
                
                self.reward += rew_seek_closer_enemy_close
            else:
                
                self.reward += rew_avoid_closer_enemy_close


        state = [self.animal.xcor(), self.animal.ycor(),
                (self.animal.xcor()-self.foods[ind_f].xcor())/240,
                (self.animal.ycor()-self.foods[ind_f].ycor())/240]
        

        
        for ind in indices:
            
            if 0 <= distin_en[ind] <= 30:
                
                if np.abs(self.animal.distance(x_in[ind], y_in[ind])) <= distin_en[ind]:
                    
                    self.reward += rew_seek_enemy_close
                else:
                    self.reward += rew_avoid_enemy_close
                    
            if 30 <= distin_en[ind] <= 65:
                
                if np.abs(self.animal.distance(x_in[ind], y_in[ind])) <= distin_en[ind]:
                    
                    self.reward += rew_seek_enemy_far
                else:
                    self.reward += rew_avoid_enemy_far
            
            self.enemies[ind].color('green')
            #if np.abs(self.animal.distance(self.enemies[ind].xcor(), self.enemies[ind].ycor())) < 20:
            #    self.enemies[ind].color('blue')
        
            
                
            state.extend([(self.animal.xcor()-self.enemies[ind].xcor())/240, 
                            (self.animal.ycor()-self.enemies[ind].ycor())/240, self.enemies[ind].dir])

            
        score = env.score
        self.run_frame()
        

       
        #print(self.enemies[ind].dir)
        #print(self.reward)
        #print(state)
        #print('Reward: ', self.reward)
        #time.sleep(2)
        #print(self.done)
        return state, self.reward, self.done, score
    
    
    def step_good(self, action):
        
        self.done = 0
        self.reward = 0
        dist_in = []
        dist_fin = []
        distin_en = []
        distfin_en = []
        dist_x = []
        dist_y = []
        
        
        for food in self.foods:
            dist_in.append(np.abs(self.animal.distance(food.xcor(), food.ycor())))
        
        
        for en in self.enemies:
            distin_en.append(np.abs(self.animal.distance(en.xcor(), en.ycor())))
            dist_x.append(self.animal.xcor()-en.xcor())
            dist_y.append(self.animal.ycor()-en.ycor())
            en.color('red')

        if action == 0:
            self.animal_left()
            
            
            if self.animal.xcor() == -240:
                self.reward -= 2
                
        if action == 1:
            self.animal_right()
            
            
            if self.animal.xcor() == +240:
                self.reward -= 2
            
        if action == 2:
            self.animal_down()
            
            
            if self.animal.ycor() == -240:
                self.reward -= 2
                
        if action == 3:
            self.animal_up()
        
            if self.animal.ycor() == +240:
                self.reward -= 2
                
        ind_f = np.argmin(dist_in)
        ind_e = np.argmin(distin_en)        
        #self.enemies[ind_e].color('green')
        x_en = self.enemies[ind_e].xcor()
        y_en = self.enemies[ind_e].ycor()
        
        self.movement()
        self.move_enemies()

        
        for food in self.foods:
            dist_fin.append(np.abs(self.animal.distance(food.xcor(), food.ycor())))
            
        for en in self.enemies:
            distfin_en.append(np.abs(self.animal.distance(en.xcor(), en.ycor())))
            
        for d in dist_fin:
            if d == 0.:
                self.reward += 12
                #self.n = 0
                
        for d in distfin_en:
            if d == 0.:
                self.reward -= 100
                self.done = 1.
                
        
        indices = sorted(range(len(distin_en)), key=lambda i: distin_en[i])[:4]
        
        if dist_fin[ind_f] < dist_in[ind_f]:
            
            self.reward += 5
        else:
            self.reward -= 5

        
        if distin_en[ind_e] <= 50:
            
            if np.abs(self.animal.distance(x_en, y_en)) <= distin_en[ind_e]:
            
                self.reward -= 3
                
            else:
                
                self.reward += 3

                
        if distin_en[ind_e] in range(0,21):

            if np.abs(self.animal.distance(dist_x[ind_e], dist_y[ind_e])) <= distin_en[ind_e]:
                self.reward -= 4
            else:
                self.reward += 2

        state = [self.animal.xcor()/240, self.animal.ycor()/240,
                (self.animal.xcor()-self.foods[ind_f].xcor())/240,
                (self.animal.ycor()-self.foods[ind_f].ycor())/240]
        

        
        for ind in indices:
            
            if distin_en[ind] in range(0,21):
                
                if np.abs(self.animal.distance(dist_x[ind], dist_y[ind])) <= distin_en[ind]:
                    self.reward -= 4
                else:
                    self.reward += 4
            if distin_en[ind] in range(21,51):
                
                if np.abs(self.animal.distance(dist_x[ind], dist_y[ind])) <= distin_en[ind]:
                    self.reward -= 1.3
                else:
                    self.reward += 1.3
            
            #self.enemies[ind].color('green')
            #if np.abs(self.animal.distance(self.enemies[ind].xcor(), self.enemies[ind].ycor())) < 20:
            #    self.enemies[ind].color('blue')
            
            state.extend([(self.animal.xcor()-self.enemies[ind].xcor())/240, 
                        (self.animal.ycor()-self.enemies[ind].ycor())/240, 
                          self.enemies[ind].dir/240])
        score = env.score
        self.run_frame()
        
        del dist_in 
        del dist_fin 
        del distin_en 
        del distfin_en 
        del dist_x 
        del dist_y 

        #print(state)
        #print(self.enemies[ind].dir)
        #print(self.reward)
        #print(state)
        #print('Reward: ', self.reward)
        #time.sleep(2)
        #print(self.done)
        return state, self.reward, self.done, score

    def step_alt(self, action):
        
        self.done = 0
        self.reward = 0
        dist_in = []
        dist_fin = []
        disten = []
        disten_f = []
        x_ini = []
        y_ini = []
        enemies_inside = []
        count = 0
        
        rew_walls = 2.5
        
        for food in self.foods:
            dist_in.append(np.abs(self.animal.distance(food.xcor(), food.ycor())))
        
        
        for en in self.enemies:
            
            disten.append(np.abs(self.animal.distance(en.xcor(), en.ycor())))
            x_ini.append(en.xcor())
            y_ini.append(en.ycor())
            en.color('red')
            if (np.abs(self.animal.xcor()-en.xcor()) < 51. and np.abs(self.animal.ycor()-en.ycor()) < 51.):
                count += 1
                enemies_inside.append(self.enemies.index(en))
                #print(self.enemies.index(en))
                #en.color('green')
        
        if action == 0:
            self.animal_left()
            
            
            if self.animal.xcor() == -240:
                self.reward -= rew_walls
                
        if action == 1:
            self.animal_right()
            
            
            if self.animal.xcor() == +240:
                self.reward -= rew_walls
            
        if action == 2:
            self.animal_down()
            
            
            if self.animal.ycor() == -240:
                self.reward -= rew_walls
                
        if action == 3:
            self.animal_up()
        
            if self.animal.ycor() == +240:
                self.reward -= rew_walls
                
        ind_f = np.argmin(dist_in)
        ind_e = np.argmin(disten)        
        #self.enemies[ind_e].color('green')
        x_en = self.enemies[ind_e].xcor()
        y_en = self.enemies[ind_e].ycor()
        
        
        self.movement()
        self.move_enemies()
        indices = sorted(range(len(disten)), key=lambda i: disten[i])[:4]
        
        for food in self.foods:
            dist_fin.append(np.abs(self.animal.distance(food.xcor(), food.ycor())))
            
        for en in self.enemies:
            disten_f.append(np.abs(self.animal.distance(en.xcor(), en.ycor())))
            
        for d in dist_fin:
            if d == 0.:
                self.reward += 3
                
                
        for d in disten_f:
            if d == 0.:
                self.reward -= 10
                self.done = 1.
                
        if dist_fin[ind_f] < dist_in[ind_f]:
            
            self.reward += 2
        else:
            self.reward -= 2
                

            

        if count == 1:
            
            
            for index in enemies_inside:
                
                if np.abs(self.animal.distance(x_ini[index], y_ini[index])) > disten[index]:
                    self.reward += 2.1
                    #print('allontanato')
                    
                elif np.abs(self.animal.distance(x_ini[index], y_ini[index])) < disten[index]:
                    self.reward -= 2
                    

        if count == 2:
            
            
            for index in enemies_inside:
                
                if np.abs(self.animal.distance(x_ini[index], y_ini[index])) > disten[index]:
                    self.reward += 2
                    
                elif np.abs(self.animal.distance(x_ini[index], y_ini[index])) < disten[index]:
                    self.reward -= 2
                    
        if count == 3:
            
            
            for index in enemies_inside:
                
                
                if np.abs(self.animal.distance(x_ini[index], y_ini[index])) > disten[index]:
                    self.reward += 2
                    
                elif np.abs(self.animal.distance(x_ini[index], y_ini[index])) < disten[index]:
                    self.reward -= 2
                    
        if count >= 4:
            
            if np.abs(self.animal.distance(x_en, y_en)) < disten[ind_e]:
                self.reward -= 1
            elif np.abs(self.animal.distance(x_en, y_en)) > disten[ind_e]:
                self.reward += 2
            
            
            
            for index in enemies_inside:
                
                
                if np.abs(self.animal.distance(x_ini[index], y_ini[index])) > disten[index]:
                    self.reward += 1
                    
                elif np.abs(self.animal.distance(x_ini[index], y_ini[index])) < disten[index]:
                    self.reward -= 1
                    
            
            
        
        

        state = [self.animal.xcor()/240, self.animal.ycor()/240,
                (self.animal.xcor()-self.foods[ind_f].xcor())/240,
                (self.animal.ycor()-self.foods[ind_f].ycor())/240]
        

        for ind in indices:
            
            if (np.abs(self.animal.xcor()-self.enemies[ind].xcor()) < 51. and np.abs(self.animal.ycor()-self.enemies[ind].ycor())< 51.):
                
                state.extend([(self.animal.xcor()-self.enemies[ind].xcor())/240, 
                            (self.animal.ycor()-self.enemies[ind].ycor())/240, 
                              self.enemies[ind].dir])
            else:
                state.extend([False, False, False])
            
            
            
        score = env.score
        self.run_frame()
        

       
        #print(self.enemies[ind].dir)
        #print('COUNT', count)
        #print(self.reward)
        #print('Dopo',x_ini)
        #print(state)
        #print('indici dei nemici ',enemies_inside)
        #print('Reward: ', self.reward)
        #time.sleep(3)
        #print(self.done)
        return state, self.reward, self.done, score



In [5]:
def get_model_weights(model):
    return [layer.get_weights() for layer in model.layers]

def compare_weights(weights1, weights2):
    return all(np.array_equal(w1, w2) for w1, w2 in zip(weights1, weights2))

In [6]:
class DQN:
    def __init__(self, action_space, state_space):
        self.action_space = action_space
        self.state_space = state_space
        self.epsilon = 1
        self.gamma = 0.90
        self.batch_size = 64
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.999992
        self.learning_rate = 0.001
        self.memory = deque(maxlen=10000)
        self.deaths = deque(maxlen=1000)
        self.model = self.build_model()
        self.target_model = self.build_model()
        self.update_target_network()
        self.losses = 0

    def build_model(self):
        model = Sequential()
        model.add(Dense(64, input_shape=(self.state_space,), activation='tanh'))
        model.add(Dense(128, activation='tanh'))
        model.add(Dense(128, activation='tanh'))
        model.add(Dense(64, activation='tanh'))
        model.add(Dense(self.action_space, activation='linear'))
        model.compile(loss='huber_loss', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def update_target_network(self):
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def remember_prioritized(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        if done:
            self.deaths.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_space)
        act_values = self.model.predict(state, verbose=0)
        return np.argmax(act_values[0])

    def act_trained(self, state):
 
        return np.argmax(self.model.predict(state, verbose=0)[0])

    def replay(self):
        if len(self.memory) < self.batch_size:
            return

        minibatch = random.sample(self.memory, self.batch_size)
        states = np.array([i[0] for i in minibatch])
        actions = np.array([i[1] for i in minibatch])
        rewards = np.array([i[2] for i in minibatch])
        next_states = np.array([i[3] for i in minibatch])
        dones = np.array([i[4] for i in minibatch])

        states = np.squeeze(states)
        next_states = np.squeeze(next_states)

        target_q_values = rewards + self.gamma * np.amax(self.target_model.predict_on_batch(next_states), axis=1) * (1 - dones)
        q_values = self.model.predict_on_batch(states)

        ind = np.array([i for i in range(self.batch_size)])
        q_values[[ind], [actions]] = target_q_values

        Hist = self.model.fit(states, q_values, epochs=1, verbose=0)
        self.losses = Hist.history['loss'][0]
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def replay_prioritized(self):
        p = 0.1
        if len(self.memory) < self.batch_size:
            return

        minibatch = []
        for i in range(self.batch_size):
            if (len(self.deaths) > 100) and (random.uniform(0, 1) < p):
                minibatch.append(random.sample(self.deaths, 1)[0])
            else:
                minibatch.append(random.sample(self.memory, 1)[0])

        states = np.array([i[0] for i in minibatch])
        actions = np.array([i[1] for i in minibatch])
        rewards = np.array([i[2] for i in minibatch])
        next_states = np.array([i[3] for i in minibatch])
        dones = np.array([i[4] for i in minibatch])

        states = np.squeeze(states)
        next_states = np.squeeze(next_states)

        target_q_values = rewards + self.gamma * np.amax(self.target_model.predict_on_batch(next_states), axis=1) * (1 - dones)
        q_values = self.model.predict_on_batch(states)

        ind = np.array([i for i in range(self.batch_size)])
        q_values[[ind], [actions]] = target_q_values

        Hist = self.model.fit(states, q_values, epochs=1, verbose=0)
        self.losses = Hist.history['loss'][0]
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


    def save_agent(self, file_path):
        # Save model
        self.model.save(file_path + "_model.h5")

        # Save optimizer state
        symbolic_weights = getattr(self.model.optimizer, 'weights')
        weight_values = tf.keras.backend.batch_get_value(symbolic_weights)
        with open(file_path + '_optimizer.pkl', 'wb') as f:
            pickle.dump(weight_values, f)

        # Save epsilon and other parameters
        with open(file_path + "_params.pkl", "wb") as f:
            pickle.dump({
                'epsilon': self.epsilon,
                'gamma': self.gamma,
                'batch_size': self.batch_size,
                'epsilon_min': self.epsilon_min,
                'epsilon_decay': self.epsilon_decay,
                'learning_rate': self.learning_rate
            }, f)
        print("Agent saved successfully.")

    @classmethod
    def load_agent(cls, action_space, state_space, file_path):
        # Load model
        loaded_model = tf.keras.models.load_model(file_path + "_model.h5")

        # Load optimizer state
        with open(file_path + '_optimizer.pkl', 'rb') as f:
            weight_values = pickle.load(f)

        # Load epsilon and other parameters
        with open(file_path + "_params.pkl", "rb") as f:
            params = pickle.load(f)

        new_agent = cls(action_space=action_space, state_space=state_space)
        new_agent.model = loaded_model
        new_agent.epsilon = params['epsilon']
        new_agent.gamma = params['gamma']
        new_agent.batch_size = params['batch_size']
        new_agent.epsilon_min = params['epsilon_min']
        new_agent.epsilon_decay = params['epsilon_decay']
        new_agent.learning_rate = params['learning_rate']

        # Set optimizer weights
        new_agent.model.optimizer.set_weights(weight_values)
        print("Agent loaded successfully with restored optimizer state.")

        return new_agent
    @classmethod
    

    def load_agent_nopar(cls, action_space, state_space, file_path):
        
        loaded_model = tf.keras.models.load_model(file_path + "_model.h5")

        new_agent = cls(action_space=action_space, state_space=state_space)

        new_agent.model = loaded_model

        return new_agent

    
    
    
def log_training_progress(episode, loss, epsilon, loss_history):
    print("Episode: {:5d}, Loss: {:.5f}, Epsilon: {:.5f}".format(episode, loss, epsilon))
    loss_history.append(loss)

def save_step_count(log_dir, step):
    step_file = os.path.join(log_dir, 'step_count.json')
    with open(step_file, 'w') as f:
        json.dump({'step': step}, f)

def load_step_count(log_dir):
    step_file = os.path.join(log_dir, 'step_count.json')
    if os.path.exists(step_file):
        with open(step_file, 'r') as f:
            data = json.load(f)
            return data['step']
    return 0

def train_dqn(episode, save_interval=50, target_update_freq=5):
    rew_history = []
    score_history = []
    epsilon_history = []
    target_updates = []
    Sc = 0

    action_space = 4
    state_space = 16
    max_steps = 1000

    agent = DQN(action_space, state_space)

    log_dir = 'tb10'
    summary_writer = tf.summary.create_file_writer(log_dir)
    start_step = load_step_count(log_dir)

    for e in range(episode):
        state = env.reset_good()
        state = np.reshape(state, (1, state_space))
        score = 0
        episode_losses = []  # To accumulate losses for the episode

        for i in range(max_steps):
            action = agent.act(state)
            next_state, reward, done, score_info = env.step_good(action)
            score += reward
            next_state = np.reshape(next_state, (1, state_space))
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            agent.replay()
            episode_losses.append(agent.losses)  
            if env.score != 0:
                Sc = env.score

            if done:
                print("episode: {}/{}, score: {}".format(e, episode, score))
                break
        
        average_loss = np.mean(episode_losses) 
        step = start_step + e
        with summary_writer.as_default():
            tf.summary.scalar('reward', score, step=step)
            tf.summary.scalar('epsilon', agent.epsilon, step=step)
            tf.summary.scalar('score', Sc, step=step)  
            tf.summary.scalar('Huber loss', average_loss, step=step)  

        score_history.append(Sc)
        rew_history.append(score)
        epsilon_history.append(agent.epsilon)
        Sc = 0

        if e % save_interval == 0:
            save_path = "models/episode_{}.h5".format(e)
            agent.save_agent(save_path)
            print("Saved model at episode {} to {}".format(e, save_path))

        if e % target_update_freq == 0:
            agent.update_target_network()
            target_updates.append(e)
            print("Updated target network at episode {}".format(e))

        save_step_count(log_dir, step)

    summary_writer.close()

    return agent, rew_history

def train_dqn_prioritized(episode, save_interval=100):
    rew_history = []
    score_history = []
    epsilon_history = []
    Sc = 0

    action_space = 4
    state_space = 16
    max_steps = 1000

    agent = DQN(action_space, state_space)

    log_dir = 'tb4_prioritized'
    summary_writer = tf.summary.create_file_writer(log_dir)
    start_step = load_step_count(log_dir)

    for e in range(episode):
        state = env.reset()
        state = np.reshape(state, (1, state_space))
        score = 0

        for i in range(max_steps):
            action = agent.act(state)
            next_state, reward, done = env.step(action)
            score += reward
            next_state = np.reshape(next_state, (1, state_space))
            agent.remember_prioritized(state, action, reward, next_state, done)
            state = next_state
            agent.replay_prioritized()
            if done:
                print("episode: {}/{}, score: {}".format(e, episode, score))
                break

        step = start_step + e
        with summary_writer.as_default():
            tf.summary.scalar('reward', score, step=step)
            tf.summary.scalar('epsilon', agent.epsilon, step=step)
            tf.summary.scalar('score', Sc, step=step) 
            tf.summary.scalar('Huber loss', agent.losses, step=step) 

        score_history.append(Sc)
        rew_history.append(score)
        epsilon_history.append(agent.epsilon)
        Sc = 0

        if e % save_interval == 0:
            save_path = "models/model_episode_{}.h5".format(e)
            agent.save_agent(save_path)
            print("Saved model at episode {} to {}".format(e, save_path))

        save_step_count(log_dir, step)

    summary_writer.close()
  

    return agent, rew_history

def train_dqn_again(episode, agent, save_interval=100, target_update_freq=4):
    rew_history = []
    score_history = []
    epsilon_history = []
    target_updates = []
    Sc = 0

    action_space = 4
    state_space = 16
    max_steps = 1000

    log_dir = 'tb10'
    summary_writer = tf.summary.create_file_writer(log_dir)
    start_step = load_step_count(log_dir)

    for e in range(episode):
        state = env.reset_good()
        state = np.reshape(state, (1, state_space))
        score = 0
        episode_losses = []  

        for i in range(max_steps):
            action = agent.act(state)
            next_state, reward, done, Score = env.step_good(action)
            score += reward
            next_state = np.reshape(next_state, (1, state_space))
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            agent.replay()
            episode_losses.append(agent.losses)  
            if env.score != 0:
                Sc = env.score

            if done:
                print("episode: {}/{}, score: {}".format(e, episode, score))
                break

        average_loss = np.mean(episode_losses)  
        step = start_step + e
        with summary_writer.as_default():
            tf.summary.scalar('reward', score, step=step)
            tf.summary.scalar('epsilon', agent.epsilon, step=step)
            tf.summary.scalar('score', Sc, step=step)
            tf.summary.scalar('Huber loss', average_loss, step=step)  

        score_history.append(Sc)
        rew_history.append(score)
        epsilon_history.append(agent.epsilon)
        Sc = 0

        if e % save_interval == 0:
            save_path = "models/model_episode_{}.h5".format(e)
            agent.save_agent(save_path)
            print("Saved model at episode {} to {}".format(e, save_path))

        if e % target_update_freq == 0:
            agent.update_target_network()
            target_updates.append(e)
            print("Updated target network at episode {}".format(e))

        save_step_count(log_dir, step)

    summary_writer.close()

    return agent, rew_history

def train_dqn_again_prioritized(episode, agent, save_interval=100, target_update_freq=4):
    rew_history = []
    score_history = []
    epsilon_history = []
    target_updates = []
    Sc = 0

    action_space = 4
    state_space = 16
    max_steps = 1000

    log_dir = 'tb10'
    summary_writer = tf.summary.create_file_writer(log_dir)
    start_step = load_step_count(log_dir)

    for e in range(episode):
        state = env.reset_good()
        state = np.reshape(state, (1, state_space))
        score = 0
        episode_losses = []  

        for i in range(max_steps):
            action = agent.act(state)
            next_state, reward, done, score_info = env.step_good(action)
            score += reward
            next_state = np.reshape(next_state, (1, state_space))
            agent.remember_prioritized(state, action, reward, next_state, done)
            state = next_state
            agent.replay_prioritized()
            episode_losses.append(agent.losses)  
            if env.score != 0:
                Sc = env.score

            if done:
                print("episode: {}/{}, score: {}".format(e, episode, score))
                break

        average_loss = np.mean(episode_losses)  
        step = start_step + e
        with summary_writer.as_default():
            tf.summary.scalar('reward', score, step=step)
            tf.summary.scalar('epsilon', agent.epsilon, step=step)
            tf.summary.scalar('score', Sc, step=step)
            tf.summary.scalar('Huber loss', average_loss, step=step)  

        score_history.append(Sc)
        rew_history.append(score)
        epsilon_history.append(agent.epsilon)
        Sc = 0

        if e % save_interval == 0:
            save_path = "models/episode_{}.h5".format(e)
            agent.save_agent(save_path)
            print("Saved model at episode {} to {}".format(e, save_path))

        if e % target_update_freq == 0:
            agent.update_target_network()
            target_updates.append(e)
            print("Updated target network at episode {}".format(e))

        save_step_count(log_dir, step)

    summary_writer.close()

    return agent, rew_history

def test_dqn(episode, agent):
    action_space = 4
    state_space = 16
    max_steps = 1000

    for e in range(episode):
        state = env.reset_good()
        state = np.reshape(state, (1, state_space))
        score = 0
        for i in range(max_steps):
            action = agent.act_trained(state)
            next_state, reward, done, Score = env.step_good(action)
            score += reward
            next_state = np.reshape(next_state, (1, state_space))
            state = next_state
            if done:
                print("episode: {}/{}, score: {}".format(e, episode, score))
                break
        
    return 

In [7]:
env = TurtleGame()

In [8]:
%tensorboard --logdir=tb10 --host "0.0.0.0" --port 60080

In [9]:
#new_agent,history = train_dqn(501, save_interval=100, target_update_freq=4)

In [10]:
#pre_save_weights = get_model_weights(new_agent.model)
#agent, loss = train_dqn(300, 100)
#new_agent.save_agent('agentmaybe')

In [11]:
agent_trained = DQN.load_agent(action_space = 4, state_space = 16, file_path = 'agentmaybe6')
#post_load_weights = get_model_weights(agent_trained.model)
#weights_are_equal = compare_weights(pre_save_weights, post_load_weights)

Agent loaded successfully with restored optimizer state.


In [12]:
agent_trained_1, loss = train_dqn_again_prioritized(401, agent_trained, save_interval = 100)

Score:  1
episode: 0/401, score: -2.0999999999999943
Agent saved successfully.
Saved model at episode 0 to models/episode_0.h5
Updated target network at episode 0
Score:  1
episode: 1/401, score: 215.10000000000008
Score:  5
episode: 2/401, score: 661.4999999999998


TurtleGraphicsError: bad color string: red

In [ ]:
agent_trained_1.save_agent('agentmaybe7')

In [ ]:
#np.mean(loss)

##### 